# Updates

### **Comments and upvotes are very much appreciated!**

**02-18-2021:**   
* added data augmentation using tfa.image  
* added checkpoint to callback list: saving the checkpoint files to kaggle's local directory (/kaggle/working) is tricky.  But I did it!
* explored additional techniques of for random image augmentation using ImageDataGenerator :  
  But I haven't been successful to make it work in the data pipeline yet.  I have commented out a few lines of codes that caused errors, noting "didn't work".   If you've got some tips to share, please leave me some comments.  :-)  
  
**02-24-2021:**  
* adjusted some data augmentation, dropout and regularization parameters.
* added checkpoint callback to the callback list.
* load the saved best model saved during training prior to computing predictions.

**02-27-20201:**
* updated and formatted quite a lot the markdown cells
* adjusted some arguments settings of data augmentation  
* explored regularization with a dropout layer and L1L2 kernel regularizer  
* explored wrapping the model with [adversarial regularization](https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist):  
`adv_config = nsl.configs.make_adv_reg_config( ... )`
`adv_model - nsl.keras.AdversarialRegularization(model, adv_config=adv_config)`
* complied the model: `adv_model.compile( ...)`
* worked on fitting the adv_model, but I kept getting errors.  So after the the efforts of wrapping my model with adversarial regularization and compiling it, I changed the cell into a markdow cell and reverted back to fitting my orginally model. If anyone can land me a hand or point me to some info, please leave me some comments.  Thanks.

**03-22-2021:**  
* added external data [Oxford Flowers TFRecords (png).](https://www.kaggle.com/cdeotte/oxford-flowers-tfrecords) Thank to Chris Deotte for converting and sharing!  
* built and trained model with DenseNet201.  
* adjusted regularization:
`tf.keras.layers.Dropout(0.3),  `
`tf.keras.layers.Dense(104, kernel_regularizer=regularizers.L2(0.0003), activation='softmax")`
* forgot to revise the total number of training images, only which resulted in swapping out some training images of this challenge with external data
* Best score = 0.94639

**03-22-2021:**  
* set up to add other external data [tf_flower_photo_tfrec (jpeg)](https://www.kaggle.com/kirillblinov/tf-flower-photo-tfrec) created from @hengck23 data. Most duplicates removed and images converted to tfrecords for convenience. Thanks to @hengck23 for creating and to Kirill Blinov for converting and sharing!   

**03-25-2021:**  
* corrected count for total number of training images  

**03-27-2021:**  
* limited external data to only [tf_flower_photo_tfrec (jpeg)](https://www.kaggle.com/kirillblinov/tf-flower-photo-tfrec) (duplicates removed): oxford_102 

**04-03-2021:**  
* removed Dropout layer  
* adjusted parameter for kernel L2 regularizar in the dense layer  


**04-04-2021:**  
* added tf.keras.layers.Dropout(0.1) **before** tf.keras.layers.GlobalAveragePooling2D()  

# Introduction

The challenge of **Petals to the Metal - Flower Classification on TPU** is to build a machine learning model that classifies 104 type of flowers in a test dataset on Kaggle.com.  Datasets and details af the challenge are on https://www.kaggle.com/c/tpu-getting-started/  After exploring different pretrained models and datasets of different image sizes, I decided to focus my efforts in building my model on DenseNet201 for the 512x512 datasets.  I tried different data augmentation techniques and other techniques to overcome overfitting.  The following lists what I have explored:  

* **Input image size**:  
  192x192  
  224x224  
  512x512  
* **Epochs**:  
  50  
  100  
  150  
  15 with EarlyStopping 
  30 with EarlyStopping
* **Pretrained model**:   
  VGG16   
  InceptionV3   
  DenseNet201   
  EfficientNetB7 (with autoaug)  
* **Number of trainable params**:  
  14,768,040 (VGG16 + dropout layer)  
  21,981,448 (InceptionV3 + dropout layer)   
  18,292,712 (DenseNet201 + dropout layer)  
  64,053,304 (EfficientNet7)    
* **Data Source**:  
  This Challenge: [Petals to the Metal - Flower Classification on TPU](https://www.kaggle.com/c/tpu-getting-started/data)  
  External: [tf_flower_photo_tfrec](https://www.kaggle.com/kirillblinov/tf-flower-photo-tfrec)  
* **Data augmentation**:   

  tf.image and tfa.image in data_augment function:
  ```
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_contrast(image, 0.9, 0.99)
  image = tf.image.random_brightness(image, 0.05)
  image = tf.image.random_saturation(image, 0.9, 0.99)
  image = tf.image.resize_with_crop_or_pad(image, HEIGHT + 180, WIDTH + 180)
  image = tf.image.random_crop(image, size=[*IMAGE_SIZE,3])
  image = tfa.image.mean_filter2d(image, filter_shape = 3) 
  ```
    tf.keras image augmentation layers:
  ```    
  data_aug_layers = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.RandomRotation(
          0.15, fill_mode='constant'),
      tf.keras.layers.experimental.preprocessing.RandomZoom(
          (-0.17, -0.01), fill_mode='constant'),
      tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
      tf.keras.layers.experimental.preprocessing.RandomTranslation(
          (-.15,.15), (-.15,.15), fill_mode='constant')
  ])
  ```
  ImageDataGenerator: random_transform method
  ```       
  img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=54, width_shift_range=0.15, height_shift_range=0.15,
      brightness_range=None, zoom_range=[1.0, 1.25], fill_mode='constant', 
      horizontal_flip=True, preprocessing_function=None) 

  image = img_gen.random_transform(image)
  ```
* **Dropout layer**:  
  ```
  tf.keras.layers.Dropout(0.3) 
  ```     
* **L2 kernel regularizer**: 
```
  tf.keras.layers.Dense(
      104, 
      kernel_regularizer=regularizers.l2(0.001), 
      activation='softmax'
  )
  ```                          
* **Callback list**: 

      callbacks=[lr_callback,es_callback, cp_callback] 
      
  Fine-tuned learning rate scheduler: exponential_lr  (credit to another Kaggler!)
  ```
  lr_callback = tf.keras.callbacks.LearningRateScheduler(
                    exponential_lr,verbose=True
  )  
  ```    
  EarlyStopping: 
  ```
  es_callback = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=2
  )
  ```    
  Checkpoint: 
  ```
  cp_callback = tf.keras.callbacks.ModelCheckpoint(
                    filepath=checkpoint_path,
                    save_weights_only=True, verbose=1
  )
  ```

# Install new libraries

In [ ]:
# upgrade pip if needed
!pip install --upgrade pip  # get the latest version
# install tensorflow-addons
!pip install -q -U tensorflow-addons
# install the Neural Structured Learning package
!pip install --quiet neural-structured-learning

# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed on Kaggle
# It is defined by the [kaggle/python Docker image](https://github.com/kaggle/docker-python)
import re, os
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import neural_structured_learning as nsl
from tensorflow.keras import regularizers      # mitigate overfitting 
from kaggle_datasets  import KaggleDatasets    # import kaggle data files
# Stop training when a monitored metric has stopped improving
# from tensorflow.keras.callbacks import EarlyStopping   
print("Tensorflow version " + tf.__version__)  # verify tensorflow versionis 2.x

# Detect Hardware

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. 
    # On Kaggle this is always the case.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Data Directories

In [ ]:
# Input data files are available in the read-only "kaggle/input/" directory
#   image files are in TFRecords format, each of which contains a sequeence
#   of records and can only be read sequentially.

TFRec_selected = '512x512'
for dirpath, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if TFRec_selected in dirpath: # 
            print(os.path.join(dirpath, filename))


In [ ]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved 
#   as output when you create a version using "Save & Run All" 
!gsutil cp /kaggle/input/tpu-getting-started/sample_submission.csv /kaggle/working/test.csv
for dirpath, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirpath, filename))     
print('list of entries contained in /kaggle/working/:',tf.io.gfile.listdir('/kaggle/working'))   
        
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of 
# the current session
!gsutil cp /kaggle/input/tpu-getting-started/sample_submission.csv /kaggle/temp/sample_submission.csv
for dirpath, _, filenames in os.walk('/kaggle/temp'):
    for filename in filenames:
        print(os.path.join(dirpath, filename))   

print('list of entries contained in /kaggle/temp/:',tf.io.gfile.listdir('/kaggle/temp'))    

# Set up data path

In [ ]:
GCS_DS_PATH_0 = KaggleDatasets().get_gcs_path("tpu-getting-started")  # Google Cloud Storage
print(GCS_DS_PATH_0)
print('Entries in the bucket 0:')
!gsutil ls $GCS_DS_PATH_0  # list items in the bucket 

# external data from "oxford-flowers-tfrecords"
#GCS_DS_PATH_1 = KaggleDatasets().get_gcs_path("oxford-flowers-tfrecords")  # Google Cloud Storage
#print(GCS_DS_PATH_1)
#print('Entries in the bucket 1:')
#!gsutil ls $GCS_DS_PATH_1  # list items in the bucket 

# external data from "tf-flower-photo-tfrec"
GCS_DS_PATH_2 = KaggleDatasets().get_gcs_path("tf-flower-photo-tfrec")  # Google Cloud Storage
print(GCS_DS_PATH_2)
print('Entries in the bucket 2:')
!gsutil ls $GCS_DS_PATH_2  # list items in the bucket 


# Set up parameters

In [ ]:
# parameters set for tfrecords-jpeg-512x512 TFRecord files
IMAGE_SIZE         = [512, 512] 
HEIGHT             = IMAGE_SIZE[0]
WIDTH              = IMAGE_SIZE[1]
EPOCHS             = 15
BATCH_SIZE         = 16 * strategy.num_replicas_in_sync

# gather filepaths
# tpu-getting-started data
TRAIN_FILENAMES_0  = tf.io.gfile.glob(GCS_DS_PATH_0 + '/tfrecords-jpeg-512x512/train/*.tfrec') 
VAL_FILENAMES      = tf.io.gfile.glob(GCS_DS_PATH_0 + '/tfrecords-jpeg-512x512/val/*.tfrec') 
TEST_FILENAMES     = tf.io.gfile.glob(GCS_DS_PATH_0 + '/tfrecords-jpeg-512x512/test/*.tfrec') 
# oxford-flowers-tfrecords data
#TRAIN_FILENAMES_1  = tf.io.gfile.glob(GCS_DS_PATH_1 + '/tfrecords-png-512x512/*.tfrec') 
# tf-flower-photo-tfrec data
#TRAIN_FILENAMES_2a = tf.io.gfile.glob(GCS_DS_PATH_2 + '/imagenet/tfrecords-jpeg-512x512/*.tfrec')  # pretrained data
#TRAIN_FILENAMES_2b = tf.io.gfile.glob(GCS_DS_PATH_2 + '/inaturalist_1/tfrecords-jpeg-512x512/*.tfrec')
#TRAIN_FILENAMES_2c = tf.io.gfile.glob(GCS_DS_PATH_2 + '/openimage/tfrecords-jpeg-512x512/*.tfrec')
#TRAIN_FILENAMES_2d = tf.io.gfile.glob(GCS_DS_PATH_2 + '/tf_flowers/tfrecords-jpeg-512x512/*.tfrec')
TRAIN_FILENAMES_2e = tf.io.gfile.glob(GCS_DS_PATH_2 + '/oxford_102/tfrecords-jpeg-512x512/*.tfrec')  
#TRAIN_FILENAMES_2  = TRAIN_FILENAMES_2b + TRAIN_FILENAMES_2c + TRAIN_FILENAMES_2d + TRAIN_FILENAMES_2e

# this challenge + external training images
TRAIN_FILENAMES    = TRAIN_FILENAMES_0 + TRAIN_FILENAMES_2e

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAIN_IMAGES = count_data_items(TRAIN_FILENAMES)
NUM_VAL_IMAGES   = count_data_items(VAL_FILENAMES)
NUM_TEST_IMAGES  = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH  = NUM_TRAIN_IMAGES // BATCH_SIZE
AUTO             = tf.data.experimental.AUTOTUNE


In [ ]:
# class names of flowers in the order of label idnum
CLASSES = [
    'pink primrose',        'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',      'wild geranium',         # 00-04
    'tiger lily',           'moon orchid',               'bird of paradise', 'monkshood',      'globe thistle',         # 05-09
    'snapdragon',           "colt's foot",               'king protea',      'spear thistle',  'yellow iris',           # 10-14
    'globe-flower',         'purple coneflower',         'peruvian lily',    'balloon flower', 'giant white arum lily', # 15-19
    'fire lily',            'pincushion flower',         'fritillary',       'red ginger',     'grape hyacinth',        # 20-24
    'corn poppy',           'prince of wales feathers',  'stemless gentian', 'artichoke',      'sweet william',         # 25-29
    'carnation',            'garden phlox',              'love in the mist', 'cosmos',         'alpine sea holly',      # 30-34
    'ruby-lipped cattleya', 'cape flower',               'great masterwort', 'siam tulip',     'lenten rose',           # 35-39
    'barberton daisy',      'daffodil',                  'sword lily',       'poinsettia',     'bolero deep blue',      # 40-44
    'wallflower',           'marigold',                  'buttercup',        'daisy',          'common dandelion',      # 45-49
    'petunia',              'wild pansy',                'primula',          'sunflower',      'lilac hibiscus',        # 50-54
    'bishop of llandaff',   'gaura',                     'geranium',         'orange dahlia',  'pink-yellow dahlia',    # 55-59
    'cautleya spicata',     'japanese anemone',          'black-eyed susan', 'silverbush',     'californian poppy',     # 60-64
    'osteospermum',         'spring crocus',             'iris',             'windflower',     'tree poppy',            # 65-69
    'gazania',              'azalea',                    'water lily',       'rose',           'thorn apple',           # 70-74
    'morning glory',        'passion flower',            'lotus',            'toad lily',      'anthurium',             # 75-79
    'frangipani',           'clematis',                  'hibiscus',         'columbine',      'desert-rose',           # 80-84
    'tree mallow',          'magnolia',                  'cyclamen ',        'watercress',     'canna lily',            # 85-89
    'hippeastrum ',         'bee balm',                  'pink quill',       'foxglove',       'bougainvillea',         # 90-94
    'camellia',             'mallow',                    'mexican petunia',  'bromelia',       'blanket flower',        # 95-99
    'trumpet creeper',      'blackberry lily',           'common tulip',     'wild rose'                                #100-103
]    

# Functions to handle data

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),    # shape [] means single element
        # class is missing, to be predicted flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of (image, idnum) pairs

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    
    # automatically interleaves reads from multiple file
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
    
    # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order) 
    
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    dataset = dataset.map(read_labeled_tfrecord if labeled 
                          else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

def get_validation_dataset(filenames):
    dataset = load_dataset(filenames,labeled=True, ordered=False)
    dataset = dataset.cache()
    dataset = dataset.shuffle(buffer_size=1920)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(filenames, ordered=True):  # order matters to submit predictions to Kaggle
    dataset = load_dataset(filenames, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset


# Data augmentation options  

Data augmentation is a way to reduce overfitting by randomly altering the training images as they are fit to the model for training.  Thus the images vary different at each epoch as though they are a different dataset, thus increasing the data size.  Here is more info at [TensorFlow](https://www.tensorflow.org/tutorials/images/data_augmentation#overview).


## 1. Use tf.image or tfa.image (on single image)

As shown in the data_augment(image, label) function below, tf.image is used to apply random augmentation on one image at a time:

    image = tf.image.random_flip_left_right(image) 
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_brightness(image,max_delta=0.1)
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
    image = tfa.image.mean_filter2d(image, filter_shape = 3)
    etc

This function is then used to map to a batch of images at the time images are fetched prior to model training:

    if augmentation:
        # map the data_augment function to each image in dataset prefetched during training
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)   
        

## 2. Use Keras preprocessing layers (on batch of images)

I explored a few image preprocessing layers for augmentation:

    data_aug_layers = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomRotation(
            0.125, fill_mode='constant'),
        tf.keras.layers.experimental.preprocessing.RandomZoom(
            (-0.17, -0.01), fill_mode='constant'),
        tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(
            (-.15,.15), (-.15,.15), fill_mode='constant')
    ])

* **Option 1**: Apply the preprocessing layers to the dataset at the time images are fetched prior to model training:

        if augmentation:  
            # apply data augmentation preprocessing layers in batch of images
            dataset = dataset.map(lambda image, y: (data_aug_layers(image, training=True), y))  

* **Option 2**: Make the preprocessing layers part of the model  

        with strategy.scope():    
        pretrained_model = efn.EfficientNetB7(
            weights='imagenet', 
            include_top=False ,
            input_shape=[*IMAGE_SIZE, 3]
        )
        pretrained_model.trainable = True # transfer learning
        model = tf.keras.Sequential([
            pretrained_model, 
            data_aug_layers,   # preprocessing layers as part of the model                  
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(len(CLASSES), kernel_regularizer=regularizers.L2(0.005), 
                activation='softmax')
        ])


## 3. ImageDataGenerator (on single image)
 
I am adding this third option to randomly tranform my training dataset.  This is great because one ImageDataGenerator with one random_transform method can do a lot of difference random augmentation on an image in one pass.  More info at [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator).

    tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False, samplewise_center=False,
        featurewise_std_normalization=False, samplewise_std_normalization=False,
        zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
        height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
        channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
        horizontal_flip=False, vertical_flip=False, rescale=None,
        preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
    )
     
My understanding is to first create the image.ImageDataGenerator with whatever data argumentation arguments you choose. Here is an example from TensorFlow:
 
    img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

Then, use the generator with the [random_transform method](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#random_transform) to apply a random transformation to a single image according to the pre-set arguments/ranges of the generator.  Below, input x is a 3D tensor, single image. The output is a randomly transformed version of x of the same shape.

    random_transform(
        x, seed=None
    )

# Functions for data augmentation

In [ ]:
##################### Keras preprocessing layers #####################

# create image augmentation layers
# 0.1 rotation = 360*0.15 = 36 deg
data_aug_layers = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(
        0.125, fill_mode='constant'),
])
# these layers removed:
#    tf.keras.layers.experimental.preprocessing.RandomZoom(
#        (-0.17, -0.01), fill_mode='constant')   
#    tf.keras.layers.experimental.preprocessing.RandomFlip(
#        mode='horizontal'), 
#    tf.keras.layers.experimental.preprocessing.RandomTranslation(
#        (-.15,.15),(-.15,.15), fill_mode='constant')

############# ImageDataGenerator - random transformation #############

# create an ImageDataGenerator 
# update this based on image augmenation exploration results
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=36, width_shift_range=0.15, height_shift_range=0.15,
    brightness_range=None, zoom_range=[1.0, 1.25], fill_mode='constant', 
    horizontal_flip=True, preprocessing_function=None)

# define data augmentation function with random_transform method 
# for dataset.map( ... )
def img_gen_random_transform(image, label):
    # apply random_transform method to single image
    image = img_gen.random_transform(image)
    return image, label


### tf.image; tfa.image; ImageDataGenerator random_transform method ###

# define data augmentation function, one image at a time                  
def data_augment(image,  label):
    
    # using tf.image 
    image = tf.image.random_flip_left_right(image) 
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_brightness(image, max_delta=0.1) 
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
    # these commented out:
    # Pad the image with a black, 90-pixel border
    #image = tf.image.resize_with_crop_or_pad(
    #            image, HEIGHT + 180, WIDTH + 180
    #)
    # Randomly crop to original size from the padded image
    #image = tf.image.random_crop(image, size=[*IMAGE_SIZE,3])

    # using tfa.image 
    #rdn = tf.random.normal([1], mean=0, stddev=1, dtype=tf.float32) 
    #if rdn > 2.0:  # blur 2.5% of the images (1 tail, 2 stddev above mean)
    #    image = tfa.image.mean_filter2d(image, filter_shape = 3,
    #                                   padding='constant')
    
    # using ImageDataGenerator random_transform method     
    #image = img_gen.random_transform(image)  # didn't work
        
    return image, label

# Function to get training dataset  
### With option for data augmentation

In [ ]:
# get training datatset with augmentation option
def get_training_dataset(filenames, augmentation=False):
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    if augmentation:
        # map the data_augment function across images of dthe ataset 
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
        # map the img_gen_random_transform function across images of the dataset
        #dataset = dataset.map(img_gen_random_transform, num_parallel_calls=AUTO)  # didn't work
    
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(buffer_size=1920)
    dataset = dataset.batch(BATCH_SIZE)
    
    if augmentation:
        # apply data augmentation preprocessing layers in batch of images
        dataset = dataset.map(lambda image, y: (data_aug_layers(image, training=True), y))
        
    dataset = dataset.prefetch(AUTO)  # prefetch next batch while training
    return dataset

# Functions to visualize images in batches

In [ ]:
np.set_printoptions(threshold=15, linewidth=80)

def batch_to_numpy_images_and_labels(databatch):
    images, labels = databatch
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    class_labels = []
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        class_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    else:
        for num in enumerate(numpy_labels):
            class_labels.append(CLASSES[num[1]])
    return numpy_images, class_labels

def show_images(databatch, row=6, col=8):  # row, col of subplots
    FIGSIZE = (col*3, row*3)  # 3X3 inch per image
    plt.figure(figsize=FIGSIZE)
    images, num_labl = batch_to_numpy_images_and_labels(databatch)
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.title(num_labl[j])
        plt.imshow(images[j])
    plt.show()
# 

# Image Augmentation Exploration
#### get training dataset with `augmentation=False`

In [ ]:
# get training dataset with augmentation=False
no_aug_train_set = get_training_dataset(TRAIN_FILENAMES, augmentation=False)

# Explore image augmentation: tf.image & tfa.image

In [ ]:
# function to show image with random data augmentation
def show_data_aug(image):
    ROW=len(images)
    COL=7  # 1 no-aug plus 6 aug images
    plt.figure(figsize=(COL*2,ROW*2))
    i=0
    for image in images:
        plt.subplot(ROW,COL,i*COL+1)
        plt.title('rdm flip L/R')
        plt.axis('off')  
        # augmented with random flip
        plt.imshow(tf.image.random_flip_left_right(image))       

        plt.subplot(ROW,COL,i*COL+2)
        plt.title('resize & rdm crop')
        plt.axis('off')    
        # Pad the image with a black, 90-pixel border
        image1 = tf.image.resize_with_crop_or_pad(
            image, HEIGHT + 180, WIDTH + 180
        )
        # Randomly crop to original size from the padded image
        image1 = tf.image.random_crop(image1, size=[*IMAGE_SIZE,3])
        plt.imshow(image1)

        plt.subplot(ROW,COL,i*COL+3)
        plt.title('rdm contrast')
        plt.axis('off')
        # augmented with contrast
        plt.imshow(tf.image.random_contrast(image, lower=0.8, upper=1.2))  

        plt.subplot(ROW,COL,i*COL+4)
        plt.title('rdm brightness')
        plt.axis('off')
        # augmented with brightness
        plt.imshow(tf.image.random_brightness(image, max_delta=0.1))       

        plt.subplot(ROW,COL,i*COL+5)
        plt.title('no aug')
        plt.axis('off')
        plt.imshow(image)

        plt.subplot(ROW,COL,i*COL+6)
        plt.title('rdm saturation')
        plt.axis('off')
        # augmented with saturation
        plt.imshow(tf.image.random_saturation(image, lower=0.7, upper=1.3))  

        plt.subplot(ROW,COL,i*COL+7)
        plt.title('rdm blur')
        plt.axis('off')        
        # ouput a rdm value from a normal distribtion 
        rdn = tf.random.normal([1], mean=0, stddev=1, dtype=tf.float32)              
        if rdn > 2.0:  # 2 stddev above mean  
            # blur 2.5% of the images
            # using tfa.image mean filter
            plt.imshow(
                tfa.image.mean_filter2d(image, filter_shape = 3,
                padding='constant')
            )  
        else:
            plt.imshow(image)
            
        i+=1
        
    plt.show()


In [ ]:
# Re-run these codes to get the next batch of no aug training images
no_aug_train_batch = (next(iter(no_aug_train_set.unbatch().batch(16)))) # get a batch for 
images, _ = batch_to_numpy_images_and_labels(no_aug_train_batch)

In [ ]:
# Re-run this after adjusting image augmentation settings 
#   of the show_data_aug() function
# compare no aug training images with random data augmentation
print('Training Dataset')
print('Image Augmentation with tf.image and tfa.image')
show_data_aug(images)

# Explore image augmentation: for keras.layers

In [ ]:
# Run these to visualize effects before implementing in
#     tf.keras.layers.experimental.preprocessing.Random___()
print('Training Dataset')
print('Image Augmentation with tf.keras.preprocesing.image.random ...')
ROW=len(images)
COL=4  # 1 no-aug plus 3 aug images
plt.figure(figsize=(COL*3,ROW*3))
i=0
for image in images:
    plt.subplot(ROW,COL,i*4+1)
    plt.title('no aug')
    plt.axis('off')
    plt.imshow(image)
    
    plt.subplot(ROW,COL,i*4+2)
    plt.title('rdm shift')
    plt.axis('off')
    # random shift on one numpy image tensor 
    # compared to tf.keras.layers.experimental.preprocessing.RandomTranslation(...)
    image2 = tf.keras.preprocessing.image.random_shift(
        image, wrg=0.15, hrg=0.15, row_axis=1, col_axis=2, channel_axis=2,
        fill_mode='constant'
    )    
    plt.imshow(image2) 

    plt.subplot(ROW,COL,i*4+3)
    plt.title('rdm 45-deg rotation')
    plt.axis('off')
    # random rotation on one numpy image tensor
    # compared to tf.keras.layers.experimental.preprocessing.RandomRotation(...)
    image3 = tf.keras.preprocessing.image.random_rotation(
        image, rg=45, row_axis=1, col_axis=2, channel_axis=2, fill_mode='constant'
    )
    plt.imshow(image3)

    plt.subplot(ROW,COL,i*4+4)
    plt.title('rdm zoom')
    plt.axis('off')
    # random zoom on one numpy image tensor
    # comapred to tf.keras.layers.experimental.preprocessing.RandomZoom(...)
    image4 = tf.keras.preprocessing.image.random_zoom(
        image, (.75, 1.0), row_axis=1, col_axis=2, channel_axis=2, fill_mode='constant'
    )
    plt.imshow(image4)
    i+=1
plt.show()

# Explore image augmentation: ImageDataGenerator

In [ ]:
# create an ImageDataGenerator for random transformation
explore_img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    brightness_range=None, zoom_range=[0.75, 1.0], fill_mode='constant', 
    horizontal_flip=True, preprocessing_function=None
)

print('Training Dataset')
print('Image Augmentation with random_transform method from ImageDataGenerator')
i = 0
ROW=8  # rows of subplots
COL=4  # cols of subplots
plt.figure(figsize=(COL*3.4,ROW*3))
for im in images:
    plt.subplot(ROW,COL,i*2+1)
    plt.title('no augmentation')
    plt.axis('off')
    plt.imshow(im)
    plt.subplot(ROW,COL,i*2+2)
    plt.title('rdm transorm from img_gen')
    plt.axis('off')
    plt.imshow(explore_img_gen.random_transform(im))
    i+=1
plt.show()

# Decide on final settings for image augmentation
After repeated trials of different settings for tf.image, tfa.image, tf.kersa.preprocessing.image and the random_transform method of ImageDataGenerator, I decided on the final settings for all image augmentation techniques.  Note: for demonstration purpose, some augmentations were actually implemented with keras.layers, where the final settings for augmentation were adjusted, too.  The following line get the training images from the GCS with `augmentation=True` with final augmenation setting prior to model training:   

`training_dataset = get_training_dataset(TRAIN_FILENAMES, augmentation=True)` 

# Visualization Samples of All Datasets

### observations  

I ran the following visualization multiple times to compare the augmented training images with the validation images and with the test images, and to make sure that I had enough randomness in image augmentation but had not introduced characteristics that are not part of the test dataset.  

This also allowed me to get an idea about the three datasets.  I was surprised to see a picture that looks like a bridge (no flowers), a little girl (no flowers) and a tatoo of flowers on someone's leg in the training dataset.  There is a blank (white) picture (label=70) in the validation dataset.  There is a picture of a fountain (no flowers) in the test dataset.  There are pictures with humans, pets and insects in the picture and a hand holding a flower with flowers in all datasets.  Pictures taken from top view and side views, a single flower or flowers in bundles, close-up and from far in a meadow or along a riverside, zoomed-in and cropped, etc are common in all datasets.

In [ ]:
# Get training dataset WITH all final data augmentation
training_dataset   = get_training_dataset(TRAIN_FILENAMES, augmentation=True)
validation_dataset = get_validation_dataset(VAL_FILENAMES)
test_dataset       = get_test_dataset(TEST_FILENAMES, ordered=False)  # not for prediction

In [ ]:
# you may run these lines multiple times to view different samples from the image sets
R = 7     # rows of subplots/images
C = 6     # cols of subplots/images
B = R*C   # number of images in a batch
print('Training Images WITH random data augmentation')
show_images(next(iter(training_dataset.unbatch().batch(B))), row=R, col=C)


In [ ]:
# you may run these lines multiple times to view different samples from the image sets
print('Validation Images')
show_images(next(iter(validation_dataset.unbatch().batch(B))), row=R, col=C)

## Shuffle test dataset to sample for visualiation  

    shuffle(
        buffer_size, seed=None, reshuffle_each_iteration=None
    )

`test_dataset.shuffle( ... )`below particularly randomly shuffles the test dataset for visualization.  

In [ ]:
# this needs TPU to run
# you may run these lines multiple times to view different samples from the image sets
print('Test Images - shuffled')
show_images(next(iter(test_dataset.shuffle(buffer_size=NUM_TEST_IMAGES).unbatch().batch(B))), 
            row=R, col=C)

# Build the model

In [ ]:
# get datasets for training
training_dataset   = get_training_dataset(TRAIN_FILENAMES, augmentation=True) # with augmentation
validation_dataset = get_validation_dataset(VAL_FILENAMES)
test_dataset       = get_test_dataset(TEST_FILENAMES, ordered=True)  
print('trainin dataset:    ', training_dataset)
print('validation dataset: ', validation_dataset)
print('test dataset:       ', test_dataset)

'''  layers created earlier and applied to prefetched the training data prior to fitting to the model
# make data_aug_layers part of the model
tf.config.set_soft_device_placement(True)   

# create image augmentation layers
    data_aug_layers = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomRotation(
            0.1, fill_mode='wrap'),
        tf.keras.layers.experimental.preprocessing.RandomZoom(
            (-0.17, -0.01), fill_mode='constant'),
        tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(
            (-.15,.15),(-.15,.15), fill_mode='constant')
    ])
'''  

# With pretrained model: DenseNet201
with strategy.scope():    
    pretrained_model = tf.keras.applications.DenseNet201(
        weights='imagenet', 
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrained_model.trainable = True # transfer learning
    model = tf.keras.Sequential([
        pretrained_model,
        # the following is commented out: layers applied to the training set instead 
        #data_aug_layers,                    
        tf.keras.layers.Dropout(0.1),  
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(104, kernel_regularizer=regularizers.L2(0.003), 
                              activation='softmax')
    ])

# display model summary, including all layers, the output shape and the number of parameters for each layer,  
#  the number of trainable parameters and the number of non-trainable parameters. 
model.summary()
    

# Adversarial Regularization


### wrap the model with adversarial regularization
```
adv_config = nsl.configs.make_adv_reg_config(
    multiplier=0.2,
    adv_step_size=0.05,
    adv_grad_norm = 'infinity'  # from TensorFlow tutorial
)
adv_model = nsl.keras.AdversarialRegularization(
    model,
    adv_config=adv_config
)
```
### compile the adv_model
```
adv_model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
```

# Compile the model 

`optimizer='adam'` implements the Adam algorithm with some default values set for some arguments, e.g. learning_rate.  Adam optimization is a stochastic gradient descent method.

`loss = 'sparse_categorical_crossentropy'` specifies that crossentropy metric is computed between the labels and predictions.  This metric is used when there are two or more label classes. Labels are expected to be provided as integers.  In this floser classification challenge, there are 104 different classes of flowers.  

`metrics=['sparse_categorical_accuracy']`  
Integer labels are used in the training, validation and test datasets. Thus metric is set to use sparse categorical accuracy, which calculates how often predictions matches the integer labels. 

In [ ]:
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

# Learning Rate Scheduler callback  
The [Learning Rate Scheduler callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LearningRateScheduler) gets the updated learning rate value from schedule function at the beginning of each epoch.  The schedule function takes an epoch index (integer, indexed from 0) and current learning rate (float) as inputs and returns a new learning rate as output (float).  

    tf.keras.callbacks.LearningRateScheduler(
        schedule, verbose=0
    )

In [ ]:
# define a fine-tuned schedule for the Learning Rate Scheduler 
def exponential_lr(epoch,
                  start_lr=0.00001,min_lr=0.00001,max_lr=0.00005,
                  rampup_epochs = 5, sustain_epochs = 0,
                  exp_decay = 0.8):  # original exp_decay = 0.8
    def lr(epoch, start_lr, min_lr,max_lr,rampup_epochs,sustain_epochs,
          exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr= ((max_lr-start_lr)/
                rampup_epochs * epoch + start_lr)
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr 
        else:
            lr = ((max_lr - min_lr)* exp_decay ** (epoch-rampup_epochs-sustain_epochs)
                  + min_lr)
            
        return lr
    return lr(epoch,start_lr,min_lr,max_lr,rampup_epochs,sustain_epochs,exp_decay)

# set learning rate scheduler for callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(schedule=exponential_lr,verbose=True)

# learning rate chart
epoch_rng = [i for i in range(EPOCHS)] 
y = [exponential_lr(x) for x in epoch_rng]
plt.plot(epoch_rng,y)
plt.xlim(-1, EPOCHS)

print("Learning rate schedule: start = {:.3g}; peak = {:.3g}; end = {:.3g}".format(y[0], max(y), y[-1]))

# EarlyStopping callback
Reference: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping    
The metric "val_loss' will be monitored and allows training to be stopped early when the metric stops improving.  I set patience = 2, so training will be stopped after 2 epochs with no improvement.

    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=0, verbose=0,
        mode='auto', baseline=None, restore_best_weights=False
    )

In [ ]:
# set earlystopping for callback
# Stop training when a monitored metric has stopped improving
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

# Checkpoint callback
I referenced tutorials on TensorFlow on [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) and on [Save checkpoints during training](
https://www.tensorflow.org/tutorials/keras/save_and_load#save_checkpoints_during_training)  

    tf.keras.callbacks.ModelCheckpoint(
        filepath, monitor='val_loss', verbose=0, save_best_only=False,
        save_weights_only=False, mode='auto', save_freq='epoch',
        options=None, **kwargs
    )

With `tf.keraas.callbacks.ModelCheckpoint(..., save_best_only=True, save_weights_only=False, ...)`, I saved my best model after each epoch if it is considered the best at the time during training.  With the saved model, I have two options:  
* After training, I may load my best model prior to computing predictions. (Trained model at the last epoch may not be the best.) 
* I may download my best model as pretrained base model to use in other platform.  When creating a notebook version using "Save & Run All", the model will be saved locally and preserved as output in my Kaggle working directory: `/kaggle/working/`.  I may download and use the model at another computing platform, like Google Colab. I may use the saved model as a base model to build on it by adding more layers, or make an adversarial model with it, etc., and see whether that makes a difference in the classification accuracy.

## Writing checkpoints locally from a TPU model
I spent a lot of time looking up for a solution for the UnimplementedError, so I want to capture what I found here. The following [example from Kaggle](https://www.kaggle.com/docs/tpu#tpu5a) shows an important argument for tf.keras.callbacks.ModelCheckpoint: `options=save_locally`.  And save_locally is configured to save model on `experimental_io_device='/job:localhost'`
```
save_locally   = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
checkpoints_cb = tf.keras.callbacks.ModelCheckpoint('./checkpoints', options=save_locally)
model.fit( . . . , callbacks=[checkpoints_cb])
```
Tensorflow has [API documentation](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) that explains how to set related arguments to save weights only as opposed to save model locally. It look simple Tensorflow's argument table for the argument "options": 

```
Optional tf.train.CheckpointOptions object if save_weights_only is true, or, 
optional tf.saved_model.SaveOptions object if save_weights_only is false.
```
That means the following:  

### **A)** If you **save weights only**, i.e. set `save_weights_only=True`:

```
tf.keras.callbacks.ModelCheckpoint(..., 
                                     save_weights_only=True, 
                                     options=save_locally,
                                     ...)
                                     
```
then, `options= ...` must be a **tf.train.CheckpointOptions( ... ) object**, i.e. have `save_locally` pre-defined as below:  
`save_locally = tf.train.CheckpointOptions(experimental_io_device='/job:localhost')`  

Note: if not saving weights locally, set `options=None`   

### **B)** If you s**ave the model**, i.e. set `save_weights_only=False`:
```
tf.keras.callbacks.ModelCheckpoint(...,
                                   save_weights_only=False`,
                                   options=save_locally,
                                   ...)
```                                   
then, `options= ...` must be a **tf.saved_model.SaveOptions( ... ) object**, i.e. have `save_locally` pre-defined as below:  
`save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')`


If options to save locally is not set up properly, the following error will occur when checkpoint files are to be saved during training:  
```
UnimplementedError - File system scheme '[local]' not implemented 
(file: '/kaggle/working/checkpoint_temp/variables/variables_temp/part-00000-of-00001') 
Encountered when executing an operation using EagerExecutor. 
This error cancels all future operations and poisons their output tensors.
```

# Create checkpoint callback
The following is to save best model locally.

In [ ]:
# set file path to save model
BEST_MODEL_PATH = "/kaggle/working/model_best.h5"   # where filename='model_best.h5'
FILE_DIR = os.path.dirname(BEST_MODEL_PATH)         # /kaggle/working/

# Create a checkpoint callback that saves the best trained model locally 
#   save_best_only=True, 
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
cp_callback  = tf.keras.callbacks.ModelCheckpoint(filepath=BEST_MODEL_PATH,      
                   options=save_locally, monitor='val_loss', verbose=1,
                   save_best_only=True, save_weights_only=False, mode='min')

# show current entries saved in Kaggle output directory
print('list of entries contained in', FILE_DIR, tf.io.gfile.listdir(FILE_DIR))
# the following two lines do the same as the above
print('list of entries contained in /kaggle/working/')  
!ls {FILE_DIR}  # same as !ls "/kaggle/working/"

# Train the model - with callbacks list
Now the model is created and configured with losses and metrics with `model.compile(...)`, it is time to train the model with `model.fit()`, which outputs a History object.  

    historical = model.fit( ...,                  
                           callbacks=[..., ...]
                           )                   
                           
As the codes indicated, while the model is being trained, events are recorded into the History object named "historical".   This object's attribute, `historical.history` is then used to create plots to show accuracy and loss metrics in the next section.  

### Callbacks
The callbacks argument `callbacks=[lr_callback, es_callback, cp_callback]` allows the listed callbacks to appy during training. Here is the list of [keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks) instances.  

The cp_callback works well and saves the best model in the .h5 format.  This allows the best model to be loaded prior to doing predictions on test dataset.  (The trained model at the last epoch may not be necessarily the best model.)  

In [ ]:
# fit/train the model 
# save the History object to the variable "historical"
# save checkpoints during training
historical = model.fit(
    training_dataset, 
    steps_per_epoch=STEPS_PER_EPOCH, 
    epochs=EPOCHS, 
    validation_data=validation_dataset,
    callbacks=[lr_callback, es_callback, cp_callback]
) 

# Plots: accuracy and loss metrics


In [ ]:
# Create plots of loss and accuracy on the training and validation datasets.

acc = historical.history['sparse_categorical_accuracy']
val_acc = historical.history['val_sparse_categorical_accuracy']

loss = historical.history['loss']
val_loss = historical.history['val_loss']

epochs_range = range(1, len(historical.history['loss'])+1)

plt.figure(figsize=(14, 14))
plt.subplot(2, 1, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')

plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.show()

# Load the best model saved during training

In [ ]:
# show files in local working directories 
print('list of entries contained in', FILE_DIR, tf.io.gfile.listdir(FILE_DIR)) 

# load best model saved by cp_callback during training
model = tf.keras.models.load_model(BEST_MODEL_PATH)

# Compute predictions on the test dataset
If you are happy with the accuracy and loss charts above, it is time to use the trained model to make predictions with `model.predict(...)`.  

`np.savetxt(...)` will create a file that can be submitted to the competition.


In [ ]:
# predict probabilities and match to the most probable integer label for each image
print('Computing predictions...')
test_images_ds = test_dataset.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)  
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

# create file to submit to the competition
print('Generating submission.csv file...')
test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), 
           fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')